 정수 n 이 주어졌을 때 n 번째 피보나치 수열의 항을 계산하는 함수를 작성하시오.
 
 예)
 $fib(0) = 0$ 

 $fib(1) = 1$
 
 $fib(5) = 5$ 
 
 $fib(10) = 55$


 
 ## 첫 번째 해법 찾기
 
 FAST 방법의 첫 번째 단계는 가능한 첫 번째 해법을 찾는 것이다. 이런 종류의 문제에서는 피보나치 수열의 정의에 집중한다면 그 해법을 찾을 수 있다. 만약 정의가 확실치 않다면 인터뷰어에게 물어보면 된다. 하지만, 대부분의 사람들이 학교에서 다음의 개념을 배웠을 것이다.

$fib(n) = fib(n-1) + fib(n-2)$

$fib(0) = 0$

$fib(1) = 1$

위의 세 표현식만 사용한다면, 모든 피보나치 수열 문제를 풀 수 있다.

세 표현식을 자세히 살펴보면 이미 재귀적 함수를 정의했다는 것을 알 수 있다. $f(0), f(1)$ 이렇게 두 개의 기본 케이스가 있다. $fib(n) = fib(n-1) + fib(n - 2)$는 재귀적 호출에 해당한다. 따라서, 첫 번째 해법을 다음과 같이 작성할 수 있다.(fig.1)

In [23]:
#Fig 1. 단순한 재귀 피보나치 해법

#java

'''
//피보나치 수열을 계산한다.
// n은 0보다 크고 int 자료형으로 결과를 저장하기에 충분하다고 가정한다.

public int fib(int n) {
    if (n == 0) return 0;
    if (n == 1) return 1;
    return fib(n-1) + fib(n-2_);
}
'''

#python

def fib(n):
    if n == 0:
        return 0
    if n == 1:
        return 1
    return fib(n-1) + fib(n-2)


print(fib(6))

8


## 첫 번째 해법 분석

첫 번째 해법은 보다시피 매우 짧고 훌륭한 방법으로 문제를 풀었다. if 문을 결합한다면 단 두 줄으로도 작성 할 수 있는 코드이다. 하지만, $fib(4)$를 수행할 때 시간에 따라 어떤 값을 쌓아나가는 지 확인해 보자. (fig.2)

![title](../img/dp_1.png "fib(4)의 실행 트리")

<center>fig 2.fib(4)의 실행 트리.</center>



위의 그래프는 이 해법이 이상적이지 않다는 것을 보여주고 있다. $fib(0)$과 $fib(1)$는 상수 시간 복잡도를 가지지만 $fib(2)$는 여러 번 호출되며, 호출될 때마다 재귀적 호출을 수반한다. 

우리는 이미 재귀적 호출의 깊이(트리의 높이)가 $n$이라는 것을 알고 있으므로 수행 시간을 빠르게 계산할 수 있다. 즉, 우리는 $n-1$번 만큼 우리가 짠 함수를 호출하게 된다. 또한, 각각의 재귀적인 호출은 우리가 기본 사례 (base case)에 도달할 때까지 항상 두 번의 호출을 수반한다. 그러므로 함수의 호출 수는 $1+2+4+8 ... + 2^{n-1}$ 또는 $2^0+2^1+...+2^{n-1}$이 되며 $O(2^n)$으로 수렴한다.

너무 긴 수행시간이다!

여기에서 재귀가 사용됐기 때문에 이 문제에 동적 프로그래밍을 적용할 수 있을지도 모른다. 가능한 지 판단하기 위해서는 다음의 두 가지 조건을 충족하는 지 확인하면 된다:
</br>

<b>1. 최적 부분 구조</b>
<br/>
지금의 해법은 재귀적이라 할 수 있다. 부분 문제들 중 하나를 풀면(예: $fib(2)$), 그 결과를 이용하여 더 큰 n에 해당하는 값들을 구할 수 있기 때문이다.
<br/>

<b>2. 중복 부분 구조</b>
<br/>
$fib(2)$가 여러 번 호출되었기 때문에 $fib(2)$를 단 한 번만 계산할 수 있다면 훨씬 더 효과적일 것이다.



이 문제는 최적 부분 구조와 중복 부분 구조를 모두 가지고 있다. 따라서, 동적 프로그래밍을 이용하여 기존의 해법을 개선시킬 수 있다. 이제 다음 FAST 방법으로 나아갈 준비가 된 것이다.

## 부분 문제 확인

각 재귀 호출마다, 해당 문제를 두 개의 부분 문제로 나눈다. 그리고 최종 결과를 얻기 위해 위의 두 부분 결과들을 결합한다. 즉, $fib(n)$라는 함수를 호출할 경우 $fib(n-1)$과 $fib(n-2)$가 부분 문제들이 된다. 

주어진 문제를 풀기 위해서는 $fib(n-1)$과 $fib(n-2)$라는 두 부분 문제들의 결과가 필요하다. 또한 이 부분 문제들은 동일한 입력값으로 여러 번 호출되는 동일한 부분 문제들이다. 따라서, 우리는 이 부분 문제들의 결과를 임시 저장(cache) 할 수 있다.

그러나, 값들을 캐싱하기 전에 부분 문제들의 의미를 확실히 해야만 한다. 이 문제의 경우, $fib(n-1)$의 값이 $n-1$번 쨰 피보나치 수열의 값이라는 것을 간단히 알 수 있다. 하지만 앞으로 나올 문제들에서는 부분 문제들을 확인하는 것이 더 복잡하기 때문에 더 중요하다.

부분 문제를 확인할 수 있다면 기존의 해법에 임시 저장(cache) 기능을 더하여 하향식의 동적 프로그래밍 해법을 도출해 낼 수 있다. (figs. 3, 4)

이 방법의 수행 시간은 이제 지수가 아닌 상수 단위로 증가한다. $fib(n)$ 을 계산 할 경우 1부터 n까지의 피보나치 수열 값을 딱 한 번씩만 계산하게 된다. 따라서 $O(n)$의 수행시간을 갖게 된다. 공간 복잡도 측면에서는 캐싱을 하기 위해 $O(n)$만큼의 공간을 사용해야 한다. 하지만 이전 해법에서도 이미 $n$ 깊이의 재귀 호출을 수행하며 $O(n)$ 만큼의 공간을 사용했기 때문에 점근적(asymptotic) 공간 복잡도에는 영향을 미치지 않는다.

![title](../img/dp_2.png "fib(4)의 실행 트리")

<center>fig 4.fib(4)의 실행 트리. 캐싱되는 부분은 붉은 색으로 표시.</center>

In [24]:
#fig 3. 하향식 동적 피보나치 해법

#java

'''
//n번째 피보나치 수열을 재귀적으로 계산한다.
//부분 문제 결과들을 캐싱하여 최적화한다.

public int fib(int n) {
    
    if (n < 2) return n;
    
    // 캐시를 만들고 -1로 초기화한다.
    int[] cache = new int[n+1];
    
    for (int i = 0; i < cache.length; i++) {
    
        cache[i] = -1;
    }
    
    //캐시의 초기값을 채운다.
    cache[0] = 0;
    cache[1] = 1;
    
    return fib(n, cache);
    
  }
  
//오버로드된 프라이빗(private) 함수

private int fib(int n, int[] cache) {

    //만약 값이 캐시 안에 있다면 반환한다.
    if(cache[n] >= 0) return cache[n];
    
    //값을 계산하고 반환하기 전에 캐시에 저장한다.
    cache[n] = fib(n-1, cache) + fib(n-2, cache);
    
    return cache[n];

}
'''

#python

def fib_(n):
    if (n < 2):
        return n
    
    cache = {}
    cache[0] = 0
    cache[1] = 1
    return fib(n, cache)


def fib(n, cache):
    if n in cache :
        return cache[n]
    
    cache[n] = fib(n-1, cache) + fib(n-2, cache)
    return cache[n]

fib_(6)

8

## 해법 수정

이제 하향식 해법을 가지고 있기 때문에 그 과정을 반대로 하여 상향식으로 문제를 풀어보자. 즉, 처음(base case)부터 시작하여 결과에 도달할 때까지 각 부분 문제들의 결과를 순서대로 계산해 나가보자.

이 문제에서 처음 단계(base case)는 $fib(0)$과 $fib(1)$이다. 이 두 값을 가지고, 다음 피보나치 수열인 $fib(2) = fib(0) + fib(1)$을 계산할 수 있다. $fib(2)$의 값을 구한 후에는 $fib(3)$을 계산할 수 있다. 이처럼 연속적으로 이전의 값들을 저장하고 필요할 때 불러다 쓸 경우 $fib(n)$에 도달하게 된다.

위 방법의 코드는 꽤 단순하다. (fig.5)

이 방식은 상향식 해법이다. 0부터 n까지 반복적으로 거쳐갔기 때문에 시간복잡도는 $O(n)$이며 0부터 n까지의 1차원 배열을 만들었기 때문에 공간 복잡도 역시 $O(n)$이다. 따라서 재귀를 사용하지 않았지만 하향식 해법과 비교할 만하다. 이 코드는 이해하기 더 쉽다.

부분 문제들의 의미를 이해하고 부분 문제들을 조합하여 순차적으로 다음 해법을 구하는 법을 이해했기 때문에 코드를 작성하기 쉬웠던 것이다. 문제를 완전히 이해할 수 있다면 하향식을 상향식으로 바꾸는 것이 어렵지 않다.

대부분의 문제는 이 단계에서 해결된다. 그러나, 이 문제의 경우 더 개선된 해법이 존재한다. 연산 과정 중에 필요한 것은 가장 최근의 두 부분 문제($cache[i=1]$ 와 $cache[i=2]$)뿐이다. 따라서, $cache[0]$부터 $cache[i-3]$은 필요하지 않으므로 메모리 내에 저장될 필요가 없다.

In [ ]:
#Fig 5. 상향식 동적 피보나치 수열 해법

#java
 
'''
//반복적으로 n번째 피보나치 수열을 계산한다.

public int fib(int n) {
    if (n == 0) return 0;
    
    //캐싱을 시작한다.
    int[] cache = new int[n+10;
    cache[1] = 1;
    
    //반복적으로 캐시를 채워 나간다.
    for (int i = 2; i <= n; i++) {
        cache[i] = cache[i-1] + cache[i-2];
    }
    
    return cache[n];
}
'''

In [ ]:
#Fig 6. 최적화된 상향식 동적 피보나치 수열 해법

#java

'''
public int fib(int n) {

    if(n < 2) return n;
    int n1=1, n2=0;
    for (int i = 2; i < n; i++) {
        int n0 = n1+n2
        n2 = n1;
        n1 = n0;
    }
    
    return n1 + n2;
}
'''

## 결론